In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Visualizing the data

__Author:__ Ji Won Park (@jiwoncpark)

__Created:__ 3/08/2021

__Last run:__ 4/12/2021

__Goals:__
We visualize the input graph.

__Before_running:__
Generate the labels with explicit kappa sampling (moving the constituent halos around in each sightline field of view (FOV) multiple times and evaluating kappa at the center), e.g.
```python
kappa_sampler = CosmoDC2Raytracer(in_dir=IN_DIR,
                                  out_dir='../kappa_sampling',
                                  fov=0.85,
                                  healpix=10450,
                                  n_sightlines=1000,  # keep this small
                                  mass_cut=11.0,
                                  n_kappa_samples=1000)
kappa_sampler.parallel_raytrace()
kappa_sampler.apply_calibration()
```

So that we won't have to explicitly sample kappa every time, we will use Gaussian process regression to infer the spread in kappa for the weighted sum of masses of each sightline.

Typically, we raytrace with halos in a smaller aperture than we query to get the graph. For illustrating the galaxy-halo connection, we'll make the aperture sizes the same. (The `aperture_size` kwarg for `CosmoDC2Graph`, which expects a radius, is half the `fov` kwarg for `CosmoDC2Raytracer`, which expects a diameter.)

In [2]:
from n2j.trainval_data.raytracers.cosmodc2_raytracer import CosmoDC2Raytracer

IN_DIR = '../n2j/data'  # where raw data lies
TRAIN_HP = [10326]
N_TRAIN = 10
BATCH_SIZE = min(N_TRAIN//5, 25)
KAPPA_DIFF = 1.0  # arcsec
fov = 1.35  # diameter of fov in arcmin
    
# Use this to infer the mean kappa contribution of new sightlines
for hp in TRAIN_HP:
    train_Y_generator = CosmoDC2Raytracer(in_dir=IN_DIR,
                                          out_dir=f'../demo_Y_{hp}',
                                          # kappa_sampling_dir='../kappa_sampling',
                                          fov=fov*2.0,
                                          healpix=hp,
                                          n_sightlines=N_TRAIN,
                                          mass_cut=11.0,
                                          n_kappa_samples=1000,
                                          seed=123)  # no sampling
    train_Y_generator.parallel_raytrace(n_cores=4)
    train_Y_generator.apply_calibration()

Generated 10 sightline(s) in 0.11 min.


100%|███████████████████████████████████████████████████████████████████████████| 10/10 [02:01<00:00, 12.16s/it]


Now we build graphs for these 100 sightlines we just computed labels for.

In [ ]:
from n2j.trainval_data.graphs.cosmodc2_graph import CosmoDC2Graph
# Features to compile
features = ['ra', 'dec', 'galaxy_id', 'redshift']
features += ['ra_true', 'dec_true', 'redshift_true']
features += ['ellipticity_1_true', 'ellipticity_2_true']
features += ['bulge_to_total_ratio_i', 'ellipticity_1_bulge_true', 'ellipticity_1_disk_true',
             'ellipticity_2_bulge_true', 'ellipticity_2_disk_true', ]
features += ['shear1', 'shear2', 'convergence']
features += ['size_bulge_true', 'size_disk_true', 'size_true']
features += ['mag_{:s}_lsst'.format(b) for b in 'ugrizY']

train_XY = CosmoDC2Graph(in_dir=IN_DIR, 
                         healpixes=TRAIN_HP, 
                         raytracing_out_dirs=[f'../demo_Y_{hp}' for hp in TRAIN_HP], 
                         aperture_size=fov*0.5,
                         n_data=[N_TRAIN], 
                         features=features, 
                         stop_mean_std_early=False,
                         n_cores=4,
                         out_dir='../demo_X')

Which sightlines should we plot? Maybe we want to check out the overdense ones, with high $\kappa$.

In [ ]:
for i in range(N_TRAIN):
    if train_XY[i].y[0, 0] > 0.05:
        print(i, train_XY[i].y)

Let's now color the nodes by the redshift.

In [ ]:
pointings = train_XY.Y

In [ ]:
data_i = 43
sample_data = train_XY[data_i]
z_src = pointings.iloc[data_i]['z']
print(z_src)
z = sample_data.x[:, 3].numpy()
z[0] = z_src
plt.hist(z, bins=np.linspace(0, 2.5, 20))
plt.axvline(z_src, color='tab:red', label='Source z')
plt.legend()

In [ ]:
out_dir = '/home/jwp/stage/sl/n2j/demo_Y_10326'

In [ ]:
# Get pointings
pointings_cols = ['kappa', 'gamma1', 'gamma2']
pointings_cols += ['galaxy_id', 'ra', 'dec', 'z', 'eps']
pointings_cols.sort()
pointings_arr = np.load(os.path.join(out_dir, 'sightlines.npy'))
pointings = pd.DataFrame(pointings_arr, columns=pointings_cols)
sightline = pointings.iloc[data_i]
cos_factor = np.cos(np.deg2rad(sightline['dec']))

In [ ]:
from torch_geometric.utils import to_networkx
import torch
import networkx as nx
import matplotlib.cm


We can additionally scale the node size by the (inverse) i-band magnitude.

In [ ]:
import matplotlib

fig, ax = plt.subplots(figsize=(12, 10))
cmap = matplotlib.cm.get_cmap('jet')
sample_data = train_XY[data_i]
# Remove mag > 25.3
mag_idx = features.index('mag_i_lsst')
mask = sample_data.x[:, mag_idx] < 30
# Put ra, dec in arcsec
ra_idx = features.index('ra_true')
dec_idx = features.index('dec_true')
print(ra_idx, dec_idx)
sample_data.x[:, ra_idx] = sample_data.x[:, ra_idx]*60.0*60.0/np.cos(np.deg2rad(sample_data.x[:, 1]))  # remove cos factor
sample_data.x[:, dec_idx] = sample_data.x[:, dec_idx]*60.0*60.0
# Update sample_data
sample_data.x = sample_data.x[mask, :]
sample_data.edge_index = torch.arange(6).reshape(2, 3)
sample_networkx = to_networkx(sample_data)
n_nodes = sample_data.x.shape[0]
# Color by redshift
z = sample_data.x[:, 3].numpy()
z[0] = pointings.iloc[data_i]['z']
scaled_z = (z - z.min())/(z.max() - z.min())  # scale 0 to 1 for colormap
node_color = cmap(scaled_z)
# Make brighter nodes bigger
mag = -sample_data.x[:, -3].numpy()
mag[0] = np.mean(mag[1:])
node_size = (mag - mag.min())/(mag.max() - mag.min())*50 + 10
#node_size[0] = 50
print("min max ra", sample_data.x[:, 4].min(), sample_data.x[:, 4].max())
nx.draw(sample_networkx, pos=dict(zip(range(n_nodes), sample_data.x[:, [4, 5]].tolist())),
        width=0.2, edge_color='tab:gray', arrowsize=0.0, alpha=1.0, 
        node_color=node_color, node_size=node_size, ax=ax)
ax.scatter([0], [0], marker='x', color='k')

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=z.min(), vmax=z.max()))
sm.set_array([])
limits = plt.axis('on') # turns on axis
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
#ax.set_xlim([-40.5, 40.5])
#ax.set_ylim([-40.5, 40.5])
fig.colorbar(sm)

print(sample_data.x.shape, sample_data.edge_index.shape)
print(sample_data.y)

In [ ]:
import glob
halos_cols = ['ra', 'ra_diff', 'dec', 'dec_diff', 'z', 'dist']
halos_cols += ['eff', 'halo_mass', 'stellar_mass', 'Rs', 'alpha_Rs']
halos_cols += ['galaxy_id']
halos_cols.sort()

halos_path_fmt = os.path.join(out_dir, 'halos_los={0:07d}_id=*.npy')
halos_path = glob.glob(halos_path_fmt.format(data_i))[0]
halos_arr = np.load(halos_path)
halos = pd.DataFrame(halos_arr, columns=halos_cols)

In [ ]:
from lenstronomy.LensModel.lens_model import LensModel
from astropy.cosmology import WMAP7   # WMAP 7-year cosmology
import n2j.trainval_data.utils.raytracing_utils as ru
import n2j.trainval_data.utils.halo_utils as hu
import n2j.trainval_data.utils.coord_utils as cu


n_halos = halos.shape[0]
# Instantiate multi-plane lens model
lens_model = LensModel(lens_model_list=['NFW']*n_halos,
                       z_source=sightline['z'],
                       lens_redshift_list=halos['z'].values,
                       multi_plane=True,
                       cosmo=WMAP7,
                       observed_convention_index=[])
halos['center_x'] = halos['ra_diff']*3600.0  # deg to arcsec
halos['center_y'] = halos['dec_diff']*3600.0
nfw_kwargs = halos[['Rs', 'alpha_Rs', 'center_x', 'center_y']].to_dict('records')
uncalib_kappa = lens_model.kappa(0.0, 0.0, nfw_kwargs,
                                 diff=KAPPA_DIFF,
                                 diff_method='square')
uncalib_gamma1, uncalib_gamma2 = lens_model.gamma(0.0, 0.0, nfw_kwargs,
                                                  diff=KAPPA_DIFF,
                                                  diff_method='square')
# Log the uncalibrated shear/convergence and the weighted sum of halo masses
w_mass_sum = np.log10(np.sum(halos['eff'].values*halos['halo_mass'].values))
new_row_data = dict(idx=[data_i],
                    kappa=[uncalib_kappa],
                    gamma1=[uncalib_gamma1],
                    gamma2=[uncalib_gamma2],
                    weighted_mass_sum=[w_mass_sum],
                    )
# Optionally map the uncalibrated shear and convergence on a grid

hu.get_kappa_map(lens_model, nfw_kwargs, fov,
                 'kappa_map.npy',
                 KAPPA_DIFF,
                #x_grid=np.arange(-fov*0.5, fov*0.5, 0.1/60.0)*60.0,
                 x_grid=np.arange(-54, 54+0.1/60.0, 0.1),
                y_grid=np.arange(-fov*0.5, fov*0.5, 0.1/60.0)*60.0)

In [ ]:
halos.columns

In [ ]:
import copy
#plt.close('all')
fig, ax = plt.subplots(figsize=(10, 10))
import lenstronomy.Util.param_util as param_util

kappa_map = np.load('kappa_map.npy')
min_k = np.min(kappa_map)
print("Minmax: ", np.min(kappa_map), np.max(kappa_map))
print(np.mean(kappa_map[kappa_map<0.4]))
#kappa_map[kappa_map<0] = np.nan
#print("Number of negative pixels: ", (~np.isfinite(kappa_map)).sum())
ax.scatter(0, 0, marker='x', color='k')
#some_halos = halos[halos['eff'] < 0.5]
if False:  # plot halo locations
    plt.scatter(halos['ra_diff']*60.0*60.0, halos['dec_diff']*60.0*60.0, 
                marker='x', color='white', s=np.log10(halos['halo_mass'].values)*4)
if True:
    plt.scatter(sample_data.x[:, 4], sample_data.x[:, 5], marker='*', color='white')
cmap = copy.copy(plt.cm.viridis)
cmap.set_bad((1, 0, 0, 1))
im = ax.imshow(kappa_map, extent=[-54, 54, -fov*60.0*0.5, fov*60.0*0.5], 
               origin='lower', 
               cmap=cmap,
              #vmin=0.08,
              #vmax=1.
              )

#im = ax.imshow(phi, extent=[-fov*60.0*0.5, fov*60.0*0.5, -fov*60.0*0.5, fov*60.0*0.5], origin='lower', cmap=cmap)
#ax.set_xticks(np.linspace(-fov*60.0*0.5, fov*60.0*0.5, 10))
#ax.set_yticks(np.linspace(-fov*60.0*0.5, fov*60.0*0.5, 10))
#plt.xlabel(r"'' ")
#plt.ylabel(r"'' ")
#ax.set_xlim([-40.5, 40.5])
#ax.set_ylim([-40.5, 40.5])
ax.set_xlabel('asec')
ax.set_ylabel('asec')
fig.colorbar(im, fraction=0.046, pad=0.04)

In [ ]:
xx = np.arange(30)
yy = np.arange(20)

xx, yy = np.meshgrid(xx, yy)

z = (xx - 2*yy)**2.0
    
#grid[0, 1] = 0.0

plt.imshow(z, origin='lower')
plt.colorbar()

In [ ]:
(-2*20)**2